In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from core.experiment import Experiment, ExpRandomeLabeledData
from copy import deepcopy
from types import MethodType
DEVICE = 'cuda:0'

In [3]:
print(Experiment.args_template())

{ 
    'output_root_dir' : , 
    'num_batches' : , 
    'tag' : , 
    'eval_interval' : , 
    'num_runs' : , 
    'num_samples' : , 
    'model' : , 
    'lr_init' : , 
    'weight_decay' : , 
    'ds_train' : , 
    'ds_test' : , 
    'momentum' : , 
    'augment' : , 
    'batch_size' : , 
    'losses' : , 
    'losses_track_only' : , 
    'w_losses' : , 
    'evaluation_policies'
}


In [4]:
args = \
{ 
    'output_root_dir' : '/tmp/debug/', 
    'num_batches' : 50000, 
    'tag' : 'need_for_performance', 
    'eval_interval' : None, 
    'num_runs' : 1, 
    'num_samples' : None, 
    'model' : ("ResNet18", { 
        'compactification_cfg': ('sphere_l2_learned', {}), #/core/models/resnet.py  / core/models/util.py
        'latent_dim': None,
        'linear_cfg': ('NormedLinear', {'learn_radius': True})   
    }), 
    'lr_init' : 0.1, 
    'weight_decay' : 0.0001, 
    'ds_train' : 'cifar10_train', 
    'ds_test' : 'cifar10_test', 
    'momentum' : 0.9, 
    'augment' : 'none', # none, supcon
    'batch_size' : 256, 
    'losses' : (
        ('CrossEntropy', {'reduction': 'mean'}), # core/experiments
#           ('SupConLoss', {'temperature': 0.1}),       
#           ('SupConLossWeighted', {'temperature': 0.1, 'weight': 0.}),    
    ), 
    'losses_track_only' : (), 
    'w_losses' : None, 
    'evaluation_policies': ('linear', 'retrained_linear', 'explicit_linear')
}

In [ ]:
def post_batch_hook(self):
    if self.batch_i % 10 == 0:
#         norm_w = torch.norm(self.model.cls.weight.data, p=2, dim=1)
#         self.logger.log_value('norm_w', norm_w)  
        self.logger.log_value('r_projection', self.model.feat_ext[-1]._r.data)
        self.logger.log_value('r_linear', self.model.cls._r.data)
    

with torch.cuda.device(DEVICE):
    exp = Experiment(**args)
#     exp.post_batch = MethodType(post_batch_hook, exp)
    exp()

ResNet(
  (feat_ext): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  